** Some pactice for CNN and RNN **

In [1]:
import tensorflow as tf
import numpy as np
from random import shuffle
from tensorflow.contrib import rnn
tf.logging.set_verbosity(tf.logging.INFO)
tf.reset_default_graph()
from IPython.core.display import Image, display
from collections import deque

# Use CNN to train MNIST

### Define estimators functions using `tf.estimator`

In [2]:
def GenerateEstimatorSpec(logits, labels, mode):
    # Generate Predictions
    predictions = {
      "classes": tf.argmax(input = logits, axis=1),
      "probabilities": tf.nn.softmax(logits, name="softmax_tensor")
    }
    print(tf.argmax(input = logits, axis=1))
    # If during training mode, just return the predictions
    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode = mode, 
                                          predictions = predictions)
    
    # If during Train or Eval, calculate cross-entropy loss
    loss = tf.losses.sparse_softmax_cross_entropy(labels = labels, 
                                                  logits = logits)

    # If during Train, update gradients
    if mode == tf.estimator.ModeKeys.TRAIN:
        optimizer = tf.train.GradientDescentOptimizer(learning_rate = 0.001)
        train_op = optimizer.minimize(
            loss = loss,
            global_step = tf.train.get_global_step())
        return tf.estimator.EstimatorSpec(mode = mode, 
                                          loss = loss, 
                                          train_op = train_op)

    # Add evaluation metrics (for EVAL mode)
    eval_metric_ops = {
          "accuracy": tf.metrics.accuracy(
              labels = labels, 
              predictions = predictions["classes"])}
    
    return tf.estimator.EstimatorSpec(
      mode = mode, loss = loss, eval_metric_ops = eval_metric_ops) 

### Define CNN network using `layers`

In [3]:
def cnn_model_fn(features, labels, mode, params = None): #params will be used for hyper params
    
    # features: This is the first item returned from the input_fn passed to train
    # labels: This is the second item returned from the input_fn passed to train
    
    #input layer (reshape from 784 cells)
    input_layer = tf.reshape(features['x'], [-1, 28, 28, 1]) 
    #[batch_size, image_width, image_height, channels], -1: dynamic compute
    
    # Convolutonal Layer 1
    conv1 = tf.layers.conv2d(
        inputs = input_layer,
        filters = 32, #number of filters
        kernel_size = [5,5], #filter size, or kernel_size = 5
        strides=(1, 1), #move along 2 dirs
        padding = "same", # other options: "valid"
        activation = tf.nn.relu,
        name = "CL_1"
        )
    
    # Note the output dimension: 28 * 28 * 32
    
    # Pooling Layer 1
    pool1 = tf.layers.max_pooling2d(inputs = conv1, 
                                    pool_size=[2, 2], 
                                    strides=2)
    
    # Note the output dimension: 14 * 14 * 32
    
    # Convolutonal Layer 2
    conv2 = tf.layers.conv2d(
        inputs = pool1,
        filters = 64, #number of filters
        kernel_size = [5,5], #filter size
        padding = "same", # other options: "valid"
        activation = tf.nn.relu,
        name = "CL_2"
    )
    
    # Note the output dimension: 14 * 14 * 64
    
    # Pooling Layer 2
    pool2 = tf.layers.max_pooling2d(inputs = conv2, 
                                    pool_size=[2, 2], 
                                    strides=2)
    
    # Note the output dimension: 7 * 7 * 64
    
    # Flatten layer (same below??)
    pool2_flat = tf.reshape(pool2, [-1, 7 * 7 * 64]) # 3136 weights
    pool2_flat = tf.layers.flatten(pool2)
    
    # FC layer (dense layer)
    dense = tf.layers.dense(inputs = pool2_flat, 
                            units=1024, # 3136 --> 1024
                            activation = tf.nn.relu)
    
    # Drop-out layer
    dropout = tf.layers.dropout(inputs=dense, 
                                rate=0.4, 
                    training = mode == tf.estimator.ModeKeys.TRAIN)# only activate during training
    
    # Final layer (10 logits)
    logits = tf.layers.dense(inputs = dropout, units = 10)
    


    Final_EstimatorSpec = GenerateEstimatorSpec(logits, labels, mode)
    return(Final_EstimatorSpec)

### Train data

In [4]:
mnist = tf.contrib.learn.datasets.load_dataset("mnist")
train_data = mnist.train.images # Returns np.array
train_labels = np.asarray(mnist.train.labels, dtype=np.int32)
eval_data = mnist.test.images # Returns np.array
eval_labels = np.asarray(mnist.test.labels, dtype=np.int32)

Extracting MNIST-data/train-images-idx3-ubyte.gz
Extracting MNIST-data/train-labels-idx1-ubyte.gz
Extracting MNIST-data/t10k-images-idx3-ubyte.gz
Extracting MNIST-data/t10k-labels-idx1-ubyte.gz


In [5]:
train_data[0].shape

(784,)

In [6]:
train_data.shape

(55000, 784)

In [7]:
type(train_data)

numpy.ndarray

In [8]:
train_data[0].dtype

dtype('float32')

In [9]:
train_labels.shape

(55000,)

In [10]:
type(train_labels)

numpy.ndarray

In [11]:
train_labels[0].dtype

dtype('int32')

### train_input_fn

In [12]:
# Returns model input
train_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": train_data}, # features 
    y=train_labels, # targets
    batch_size = 100, 
    num_epochs = None, # forever, the model will train until the specified number of steps is reached
    shuffle = True)

### mnist_classifier

In [13]:
# Create the Estimator
mnist_classifier = tf.estimator.Estimator(
    model_fn = cnn_model_fn, # model function type, 
    model_dir = "./model_files",
    params = None)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': './model_files', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x1c1567eda0>, '_task_type': 'worker', '_task_id': 0, '_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


### logging_hook

In [14]:
# Set up logging for predictions
tensors_to_log = {"probabilities": "softmax_tensor"}
logging_hook = tf.train.LoggingTensorHook(
      tensors=tensors_to_log, every_n_iter=50)

### Start training

In [16]:
mnist_classifier.train(
    input_fn = train_input_fn,
    steps = 10) # for testing purpose
    #hooks = [logging_hook])

Tensor("ArgMax_1:0", shape=(100,), dtype=int64)
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Restoring parameters from ./model_files/model.ckpt-2
INFO:tensorflow:Saving checkpoints for 3 into ./model_files/model.ckpt.
INFO:tensorflow:loss = 2.305, step = 3
INFO:tensorflow:Saving checkpoints for 12 into ./model_files/model.ckpt.
INFO:tensorflow:Loss for final step: 2.30147.


In [17]:
# Evaluate the model and print results
eval_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": eval_data}, #features 
    y=eval_labels,
    num_epochs=1,
    shuffle=False)

eval_results = mnist_classifier.evaluate(input_fn=eval_input_fn)
print(eval_results)

Tensor("ArgMax_1:0", shape=(?,), dtype=int64)
INFO:tensorflow:Starting evaluation at 2018-04-30-00:22:46
INFO:tensorflow:Restoring parameters from ./model_files/model.ckpt-12
INFO:tensorflow:Finished evaluation at 2018-04-30-00:22:52
INFO:tensorflow:Saving dict for global step 12: accuracy = 0.1444, global_step = 12, loss = 2.30339
{'accuracy': 0.1444, 'loss': 2.303391, 'global_step': 12}


# Use RNN (`LSTM`) to train `sum` operation

### Define RNN network

In [19]:
def rnn_model_fn(features, labels, mode, params = None): #params will be used for hyper params
    
    num_time_steps = 12;
    num_dims = 1;
    num_hidden_units = 24;
    num_outputs = 13;
    
    input_layer = tf.unstack(features['x'], num_time_steps, num_dims)# (data, time_steps, 1), cut into 10 pieces
    
    lstm_layer = rnn.BasicLSTMCell(num_hidden_units )
    
    outputs, _ = rnn.static_rnn(lstm_layer, input_layer ,dtype = "float32")
    
    logits = tf.layers.dense(inputs = outputs[-1], units = num_outputs)
    
    Final_EstimatorSpec = GenerateEstimatorSpec(logits, labels, mode)
    return(Final_EstimatorSpec)

### Generate synthetic training data

In [20]:
train_input = ['{0:012b}'.format(i) for i in range(2**12)] 
shuffle(train_input)
ti  = []
for i in train_input:
    temp_list = []
    for j in i:
        temp_list.append([j])
    ti.append(temp_list)

train_input = np.array(ti, dtype=np.float32)
train_input[5]

array([[ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 1.],
       [ 0.],
       [ 0.],
       [ 1.],
       [ 0.],
       [ 1.],
       [ 0.]], dtype=float32)

In [21]:
train_output = []
for i in train_input:
    count = 0
    for j in i:
        if j[0] == 1:
            count+=1
    train_output.append(count)

train_output = np.array(train_output, dtype=np.int32)

In [22]:
NUM_EXAMPLES = 4000
train_data = train_input[:NUM_EXAMPLES]
train_labels = train_output[:NUM_EXAMPLES]
eval_data = train_input[NUM_EXAMPLES:]
eval_labels = train_output[NUM_EXAMPLES:]

### Build model

In [23]:
# Returns model input
train_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": train_data}, # features 
    y=train_labels, # targets
    batch_size = 5, 
    num_epochs = None, # forever, the model will train until the specified number of steps is reached
    shuffle = True)

In [24]:
# Create the Estimator
sum_classifier = tf.estimator.Estimator(
    model_fn = rnn_model_fn, # model function type, 
    model_dir = "./model_files_2",
    params = None)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': './model_files_2', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x1c1602d2b0>, '_task_type': 'worker', '_task_id': 0, '_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [26]:
sum_classifier.train(
    input_fn = train_input_fn,
    steps = 10)

Tensor("ArgMax_1:0", shape=(5,), dtype=int64)
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Restoring parameters from ./model_files_2/model.ckpt-60002
INFO:tensorflow:Saving checkpoints for 60003 into ./model_files_2/model.ckpt.
INFO:tensorflow:loss = 0.799017, step = 60003
INFO:tensorflow:Saving checkpoints for 60012 into ./model_files_2/model.ckpt.
INFO:tensorflow:Loss for final step: 0.878255.


### Evaluation

In [27]:
# Evaluate the model and print results
eval_input_fn = tf.estimator.inputs.numpy_input_fn(
    x = {"x": eval_data}, #features 
    y = eval_labels,
    num_epochs = 1,
    shuffle = False)

eval_results = sum_classifier.evaluate(input_fn=eval_input_fn)
print(eval_results)

Tensor("ArgMax_1:0", shape=(?,), dtype=int64)
INFO:tensorflow:Starting evaluation at 2018-04-30-00:25:22
INFO:tensorflow:Restoring parameters from ./model_files_2/model.ckpt-60012
INFO:tensorflow:Finished evaluation at 2018-04-30-00:25:22
INFO:tensorflow:Saving dict for global step 60012: accuracy = 0.864583, global_step = 60012, loss = 0.611921
{'accuracy': 0.86458331, 'loss': 0.61192131, 'global_step': 60012}


### Prediction

In [33]:
# Predict the model and print results
pred_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": eval_data}, #features 
    y=eval_labels,
    num_epochs=1,
    shuffle=False)

In [34]:
pred_results = sum_classifier.predict(input_fn=pred_input_fn)

In [35]:
pred_classes = []
for i in pred_results:
    pred_classes.append(i['classes'])
print(pred_classes[:10])

Tensor("ArgMax_1:0", shape=(?,), dtype=int64)
INFO:tensorflow:Restoring parameters from ./model_files_2/model.ckpt-60012
[5, 7, 6, 7, 4, 8, 6, 7, 6, 5]


# Use RNN(LSTM) to train `sum` Operation

A Straight forward example to predict many-to-one problem

how to predict '[0,1,1,0]' to 2 (SUM)

http://monik.in/a-noobs-guide-to-implementing-rnn-lstm-using-tensorflow/

In [28]:
import numpy as np
import random
from random import shuffle
from tensorflow.contrib import rnn
import tensorflow as tf
tf.reset_default_graph()

In [29]:
train_input = ['{0:010b}'.format(i) for i in range(2**10)]
shuffle(train_input)
train_input = [map(int,i) for i in train_input]
ti  = []
for i in train_input:
    temp_list = []
    for j in i:
            temp_list.append([j])
    ti.append(np.array(temp_list))
train_input = ti

train_input[0]

array([[1],
       [0],
       [1],
       [1],
       [0],
       [1],
       [0],
       [0],
       [0],
       [1]])

In [30]:
train_output = []
for i in train_input:
    count = 0
    for j in i:
        if j[0] == 1:
            count+=1
    temp_list = ([0]*11)
    temp_list[count]=1
    train_output.append(temp_list)

train_output[0] # one-hot

[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0]

In [31]:
NUM_EXAMPLES = 900
test_input = train_input[NUM_EXAMPLES:]
test_output = train_output[NUM_EXAMPLES:]
train_input = train_input[:NUM_EXAMPLES]
train_output = train_output[:NUM_EXAMPLES]

In [32]:
data.shape

TensorShape([Dimension(None), Dimension(10), Dimension(1)])

In [33]:
data = tf.placeholder(tf.float32, [None, 10, 1]) # Batch_size, Timestep, dimension of each input
input = tf.unstack(data, 10, 1)# (data, time_steps, 1), cut into 10 pieces
y = tf.placeholder(tf.float32, [None, 11])

num_hidden = 24
lstm_layer = rnn.BasicLSTMCell(num_hidden)
outputs, _ = rnn.static_rnn(lstm_layer, input,dtype = "float32")

out_weights = tf.Variable(tf.random_normal([num_hidden, 11]))
out_bias = tf.Variable(tf.random_normal([11]))
prediction = tf.matmul(outputs[-1], out_weights)+ out_bias

loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits = prediction, labels = y))
opt = tf.train.AdamOptimizer(learning_rate = 0.001).minimize(loss)
correct_prediction = tf.equal(tf.argmax(prediction, 1), tf.argmax(y,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [ ]:
init_op = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init_op)

batch_size = 10
no_of_batches = int(len(train_input) / batch_size)
epoch = 100
for i in range(epoch):
    ptr = 0
    for j in range(no_of_batches):
        inp, out = train_input[ptr:ptr+batch_size], train_output[ptr:ptr+batch_size]
        ptr+=batch_size
        acc, _ = sess.run([loss, opt],feed_dict={data: inp, y: out})
        print(acc)

acc = sess.run(accuracy,{data: test_input, y: test_output})
print(sess.run(prediction,{data: [[[1],[0],[0],[1],[1],[0],[1],[1],[1],[0]]]}))
print('Epoch {:2d} acc {:3.1f}%'.format(i + 1, 100 * acc))
sess.close()